In [65]:
import pandas as pd
import os
import time
import shutil
import utils, models
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt
import datetime
# from tensorboard_logger import Logger
import json
import confuse_matrix as cm
np.seterr(divide='ignore',invalid='ignore')


{'divide': 'ignore', 'over': 'warn', 'under': 'ignore', 'invalid': 'ignore'}

In [66]:
def dataset_generate(directory):
    # '''------------数据预处理相关--------------'''
    # 加载数据集并分割数据测试集和验证集

    # 训练集

    on00_x = np.zeros((30,200,9))
    for i in range(np.shape(on00_x)[0]):
        on00_x[i,:,:] = np.loadtxt('/home/nuci7/project/cf2/crazyflie-firmware/control/train_data/on00_'+str(i+1)+'.txt')
    on00_y = 0 * np.ones(on00_x.shape[0])


    on45_x = np.zeros((30,200,9))
    for i in range(np.shape(on45_x)[0]):
        on45_x[i,:,:] = np.loadtxt('/home/nuci7/project/cf2/crazyflie-firmware/control/train_data/on45_'+str(i+1)+'.txt')
    on45_y = 1 * np.ones(on45_x.shape[0])

    on135_x = np.zeros((30,200,9))
    for i in range(np.shape(on135_x)[0]):
        on135_x[i,:,:] = np.loadtxt('/home/nuci7/project/cf2/crazyflie-firmware/control/train_data/on135_'+str(i+1)+'.txt')
    on135_y = 2 * np.ones(on135_x.shape[0])


    # epoxy_plate_x = np.load(train_val_save_dir + "epoxy-plate.npy")
    # epoxy_plate_y = 3 * np.ones(epoxy_plate_x.shape[0])

    # glass_x = np.load(train_val_save_dir + "glass.npy")
    # glass_y = 4 * np.ones(glass_x.shape[0])
    #     # 训练集
    # leather_x = np.load(train_val_save_dir+"leather.npy")
    # leather_y = 5 * np.ones(leather_x.shape[0])

    # lint_x = np.load(train_val_save_dir+"lint.npy")
    # lint_y = 6 * np.ones(lint_x.shape[0])

    # nylon_x = np.load(train_val_save_dir+"nylon.npy")
    # nylon_y = 7 * np.ones(nylon_x.shape[0])

    # silicon_x = np.load(train_val_save_dir + "silicon.npy")
    # silicon_y = 8* np.ones(silicon_x.shape[0])

    # wood_x = np.load(train_val_save_dir + "wood.npy")
    # wood_y = 9* np.ones(wood_x.shape[0])



    X = np.concatenate((on00_x [0:30, :],
                        on45_x [0:30, :],
                        on135_x [0:30, :],),
                        axis = 0)
    y = np.concatenate((on00_y [0:30],
                        on45_y [0:30],
                        on135_y [0:30],),
                        axis = 0)

    X = np.delete(X, [7,8], axis=2)
    X = X.reshape((X.shape[0],7, X.shape[1]))
    print(X.shape)
    print(y.shape)

    on00_x = X[0:30]
    on45_x = X[30:60]
    on135_x = X[60:]


    perm = np.random.permutation(X.shape[0])
    X_shuffled = X[perm]
    y_shuffled = y[perm]



    # # 打乱顺序
    # X_random = np.zeros((X.shape[0], X.shape[1], X.shape[2]))
    # y_random = np.zeros((X.shape[0]))

    # for i in range(X.shape[0]):
    #     if i % 3 == 0:
    #         X_random[i] = on00_x [i // 10, :]
    #         y_random[i] = on00_y [i // 10]
    #     elif i % 3 == 1:
    #         X_random[i] = on45_x [i // 10, :]
    #         y_random[i] = on45_y [i // 10]
    #     elif i % 3 == 2:
    #         X_random[i] = on135_x [i // 10, :]
    #         y_random[i] = on135_y [i // 10]

    # shuffle_indices = np.random.permutation(np.arange(len(y_random)))
    # # X_shuffled = X_random[shuffle_indices]
    # # y_shuffled = y_random[shuffle_indices]

    # X_shuffled = X_random
    # y_shuffled = y_random

    print(X_shuffled.shape)
    print(y_shuffled.shape)
        # 数据归一化
    for i in range(X_shuffled.shape[0]):
        X_shuffled[i,0] = X_shuffled[i,0] / X_shuffled[i,0].max(axis=0).reshape(-1,1)

    # 将numpy数据封装为pytorch上的.pth文件
    X_train = X_shuffled[0:70, :,:].astype('float32')
    y_train = y_shuffled[0:70].astype('float32')


    X_val = X_shuffled[70:, :].astype('float32')
    y_val = y_shuffled[70:].astype('float32')

    dd = {'train': X_train,
            'val': X_val,
            'label_train': y_train,
            'label_val': y_val}
    # 保存数据集为.pth文件
    print(np.shape(X_val))

    torch.save(dd, directory)


In [67]:
def save_ckpt(state, is_best, model_save_dir):
    current_w = os.path.join(model_save_dir, 'current_w.pth')
    best_w = os.path.join(model_save_dir, 'best_w.pth')
    torch.save(state, current_w)
    if is_best: shutil.copyfile(current_w, best_w)


class TENGDataset(Dataset):
    """
    A generic data loader where the samples are arranged in this way:
    dd = {'train': train, 'val': val, "idx2name": idx2name, 'file2idx': file2idx}
    """

    def __init__(self, path, train=True):
        super(TENGDataset, self).__init__()
        dd = torch.load(path)
        self.train = train
        self.data = dd['train'] if train else dd['val']
        self.label = dd['label_train'] if train else dd['label_val']

    def __getitem__(self, index):
        x = self.data[index].astype('float32')
        target = self.label[index]
        return x, target

    def __len__(self):
        return len(self.data)


def train_epoch(model, optimizer, train_dataloader, show_interval=10):
    model.train()
    acc_meter, loss_meter, it_count = 0, 0, 0
    for inputs, target in tqdm(train_dataloader):
        inputs = inputs.to(device)
        target = target.to(device)
        optimizer.zero_grad()
        output = F.log_softmax(model(inputs), dim=1)
#         print(torch.exp(output[0]))
#         print(target[0])
        loss = F.nll_loss(output, target.long())
        loss.backward()
        optimizer.step()
        loss_meter += loss.item()
        pred = output.argmax(dim=1)
        acc = torch.eq(pred, target).sum().float().item() / len(inputs)
        acc_meter += acc
        it_count += 1
#         if it_count != 0 and it_count % show_interval == 0:
#             print("%d, loss: %.3e acc: %.3f" % (it_count, loss.item(), acc))
    return loss_meter / it_count, acc_meter / it_count


def val_epoch(model, optimizer, val_dataloader):
    model.eval()
    acc_meter, loss_meter, it_count = 0, 0, 0
    with torch.no_grad():
        for inputs, target in val_dataloader:
            inputs = inputs.to(device)
            target = target.to(device)
            optimizer.zero_grad()
            output = F.log_softmax(model(inputs), dim=1)
            loss = F.nll_loss(output, target.long())
            loss_meter += loss.item()
            pred = output.argmax(dim=1)
            acc = torch.eq(pred, target).sum().float().item() / len(inputs)
            acc_meter += acc
            it_count += 1
    return loss_meter / it_count, acc_meter / it_count

# 训练结果可视化 准确率+损失
def plot_accuracies_loss(history, figs_directory):
    # 设置字体
    SMALL_SIZE = 8
    MEDIUM_SIZE = 10
    BIGGER_SIZE = 12
    plt.rc('font', size=BIGGER_SIZE)  # controls default text sizes
    plt.rc('axes', titlesize=BIGGER_SIZE)  # fontsize of the axes title
    plt.rc('axes', labelsize=BIGGER_SIZE)  # fontsize of the x and y labels
    plt.rc('xtick', labelsize=SMALL_SIZE)  # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)  # fontsize of the tick labels
    plt.rc('legend', fontsize=BIGGER_SIZE)  # legend fontsize
    plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
    Validation_accuracies = [x['val_accuracy'] for x in history]
    Training_Accuracies = [x['train_accuracy'] for x in history]
    train_losses = [x['train_loss'] for x in history]
    val_losses = [x['val_loss'] for x in history]
    plt.plot(Training_Accuracies, '-ro',linewidth=1)
    plt.plot(Validation_accuracies, '-bo',linewidth=1)
    plt.plot(train_losses, '-rx')
    plt.plot(val_losses, '-bx')
    plt.xlabel('Number of epochs')
    plt.ylabel('accuracy/loss')
    plt.legend(['Training_Accuracies', 'Validation_accuracies','train_losses','val_losses'])
    plt.title('Accuracy/Loss vs. Number of epochs')
    plt.savefig(figs_directory+'-plot_accuracies_loss.jpg', dpi=1000)  # 指定分辨率保存


# 训练结果可视化 准确率+损失
def plot_accuracies(history, figs_directory):
    # 设置字体
    SMALL_SIZE = 8
    MEDIUM_SIZE = 10
    BIGGER_SIZE = 12
    plt.rc('font', size=BIGGER_SIZE)  # controls default text sizes
    plt.rc('axes', titlesize=BIGGER_SIZE)  # fontsize of the axes title
    plt.rc('axes', labelsize=BIGGER_SIZE)  # fontsize of the x and y labels
    plt.rc('xtick', labelsize=SMALL_SIZE)  # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)  # fontsize of the tick labels
    plt.rc('legend', fontsize=BIGGER_SIZE)  # legend fontsize
    plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
    

    Validation_accuracies = [x['val_accuracy'] for x in history]
    Training_Accuracies = [x['train_accuracy'] for x in history]
    plt.plot(Training_Accuracies, '-ro',linewidth=1)
    plt.plot(Validation_accuracies, '-bo',linewidth=1)
    plt.xlabel('Number of epochs')
    plt.ylabel('accuracy')
    plt.legend(['Training', 'Validation'], loc='lower right')
    plt.title('Accuracy vs. Number of epochs')
    plt.savefig(figs_directory+'plot_accuracies.jpg', dpi=1000)  # 指定分辨率保存


def plot_losses(history, figs_directory):
    # 设置字体
    SMALL_SIZE = 8
    MEDIUM_SIZE = 10
    BIGGER_SIZE = 12
    plt.rc('font', size=BIGGER_SIZE)  # controls default text sizes
    plt.rc('axes', titlesize=BIGGER_SIZE)  # fontsize of the axes title
    plt.rc('axes', labelsize=BIGGER_SIZE)  # fontsize of the x and y labels
    plt.rc('xtick', labelsize=SMALL_SIZE)  # fontsize of the tick labels
    plt.rc('ytick', labelsize=SMALL_SIZE)  # fontsize of the tick labels
    plt.rc('legend', fontsize=BIGGER_SIZE)  # legend fontsize
    plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

    train_losses = [x['train_loss'] for x in history]
    val_losses = [x['val_loss'] for x in history]
    plt.plot(train_losses, '-bx',linewidth=1)
    plt.plot(val_losses, '-rx',linewidth=1)
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.legend(['Training', 'Validation'])
    plt.title('Loss vs. No. of epochs')
    plt.savefig(figs_directory+'-plot_losses.jpg', dpi=1000)  # 指定分辨率保存


# 训练函数
def train_main(model_select="1D-CNN", inplanes=2, num_classes=4, batch_size=32, learning_rate=0.001, epoch_max=5):
    # 数据集保存位置
    directory = "./" + model_select + f'/Train_{datetime.datetime.now().strftime("%Y%m%d-%H%M")}/'
    if not os.path.exists(directory):
        os.makedirs(directory)
    data_directory = directory + f'data_{datetime.datetime.now().strftime("%Y%m%d-%H%M")}.pth'
    model_directory = directory + f'model_{datetime.datetime.now().strftime("%Y%m%d-%H%M")}'
    states_directory = directory + f'states_{datetime.datetime.now().strftime("%Y%m%d-%H%M")}.pth'
    history_directory = directory + f'history_{datetime.datetime.now().strftime("%Y%m%d-%H%M")}.csv'
    figs_directory = directory + f'/figs'
    # 数据集生成
    dataset_generate(data_directory)
    # 数据集导入
    train_dataset = TENGDataset(data_directory, train=True)
    train_dataloader = DataLoader(train_dataset, batch_size, shuffle=True, num_workers=1)
    val_dataset = TENGDataset(data_directory, train=False)
    val_dataloader = DataLoader(val_dataset, batch_size, shuffle=True, num_workers=1)
    print("train data size:", len(train_dataset), ",val data size:", len(val_dataset))

    # 模型建立
    if model_select == "1D-CNN":
        model = models.Conv1dNet(inplanes, num_classes)
    elif model_select == "ResCNN":
        model = models.ResCNN_Net(inplanes, num_classes)
    elif model_select == "LSTM-FCN":
        model = models.LSTM_FCN_Net(inplanes, 1000, num_classes)
    elif model_select == "InceptionTime":
        model = models.InceptionTime_Net(inplanes, num_classes)
    elif model_select == "FR_CNN_Net":
        model = models.FR_CNN_Net(inplanes, num_classes)
    else:
        model = models.Conv1dNet(inplanes, num_classes)

    model = model.to(device)
    model_save_dir = model_directory
    if not os.path.exists(model_save_dir):
        os.makedirs(model_save_dir)
    best_lost = 1e3
    lr = learning_rate #1e-3
    start_epoch = 0
    stage = 1
    # logger = Logger(logdir=model_save_dir, flush_secs=2)
    states = []

    optimizer = optim.SGD(model.parameters(), lr, momentum=0.9, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.ExponentialLR(optimizer, gamma=1)  # 0.95

    history = []

    for epoch in range(start_epoch, epoch_max):
        since = time.time()
        train_loss, train_acc = train_epoch(model, optimizer, train_dataloader)
        val_loss, val_acc = val_epoch(model, optimizer, val_dataloader)

        print('#epoch: %02d \n train_loss: %.3e train_acc: %.3f \n val_loss: %0.3e val_acc: %.3f time: %s\n'
              % (epoch, train_loss, train_acc, val_loss, val_acc,'56'))# utils.print_time_cost(since)))

        # logger.log_value('train_loss', train_loss, step=epoch)
        # logger.log_value('train_acc', train_acc, step=epoch)
        # logger.log_value('val_loss', val_loss, step=epoch)
        # logger.log_value('val_acc', val_acc, step=epoch)

        state = {"state_dict": model.state_dict(), "epoch": epoch,
                 "loss": val_loss, 'acc': val_acc, 'lr': lr, 'stage': stage}
        states.append(state)

        scheduler.step()
        # for plot result
        result = {}
        result['train_loss'] = train_loss
        result['train_accuracy'] = train_acc
        result['val_loss'] = val_loss
        result['val_accuracy'] = val_acc
        history.append(result)

        save_ckpt(state, best_lost > val_loss, model_save_dir)
        best_lost = min(best_lost, val_loss)
        if epoch in [5, 15, 25]:
            stage += 1
            lr /= 10
            current_w = os.path.join(model_save_dir, 'current_w.pth')
            model.load_state_dict(torch.load(current_w)['state_dict'])
            print("*" * 10, "step into stage%02d lr %.3ef" % (stage, lr))
            # utils.adjust_learning_rate(optimizer, lr)

    torch.save(states, states_directory)
    
    # 保存模型

    torch.save(model.state_dict(),  directory +model_select+ ".model")
    torch.save(model, directory + model_select + ".pth")
    # 保存训练过程history 到txt 文件
    # history_save_path = './History_' + datetime.datetime.now().strftime("%Y%m%d") + '/'
    # if not os.path.exists(history_save_path): 
    #     os.makedirs(history_save_path)


    # filename = open(history_directory, 'w')
    # for item in history:
    #     filename.write(str(item))
    #     filename.write("\r\n")
    # filename.close()

    dataframe = pd.DataFrame(history)
    # 将DataFrame存储为csv,index表示是否显示行名，default=True
    dataframe.to_csv(history_directory, index=False, sep=',')


    # # 训练结果可视化
    # plot_accuracies(history, figs_directory)
    # plt.show()
    # plot_losses(history, figs_directory)
    # plt.show()
    # plot_accuracies_loss(history, figs_directory)
    # plt.show()
    # plot_confusion_matrix(data_directory,directory + model_select+ ".model",model_select,figs_directory,batch_size,inplanes,num_classes,json_label_path='class_object.json')
    # plt.show()




# 测试函数
def test_main(inplanes=19, num_classes=12, state_dict_file='./Train_20211230/con1d_20211230-115750/best_w.pth', dataset_file='./Train_20211230/train_20211230-1157.pth'):
    model = models.Conv1dNet(inplanes, num_classes)
    model = model.to(device)
    model.load_state_dict(torch.load(state_dict_file)['state_dict'])
    test_dataset = torch.load(dataset_file)['val']
    test_label = torch.load(dataset_file)['label_val']
    inputs = torch.Tensor(test_dataset).to(device)
    output = F.log_softmax(model(inputs), dim=1)
    pred = output.argmax(dim=1)
    pred_new = pred.cpu().numpy()
    # pred_count = []
    # expand = 20
    # for i in range(len(pred_new) // expand):
    #     c = np.bincount(pred_new[i * expand:(i + 1) * expand])
    #     pred_count.append(np.argmax(c))
    #     print(pred_new[i * expand:(i + 1) * expand].tolist())
    print(pred_new.tolist())
    print(test_label)


# 绘制混淆矩阵
def plot_confusion_matrix(val_dataset_file, model_file, model_select,figs_directory,batch_size=32, inplanes=1, num_classes=4, json_label_path='class_material.json'):
    # 绘制混淆矩阵
    # load val dataloader
    validate_dataset = TENGDataset(val_dataset_file, train=False)
    validate_dataloader = DataLoader(validate_dataset, batch_size, shuffle=True, num_workers=1)
    print("val data size:", len(validate_dataset))

    # 模型建立
    if model_select == "1D-CNN":
        net = models.Conv1dNet(inplanes, num_classes)
    elif model_select == "ResCNN":
        net = models.ResCNN_Net(inplanes, num_classes)
    elif model_select == "LSTM-FCN":
        net = models.LSTM_FCN_Net(inplanes, 1000, num_classes)
    elif model_select == "InceptionTime":
        net = models.InceptionTime_Net(inplanes, num_classes)
    elif model_select == "FR_CNN_Net":
        net = models.FR_CNN_Net(inplanes, num_classes)
    else:
        net = models.Conv1dNet(inplanes, num_classes)

    # load pretrain weights
    model_weight_path = model_file
    assert os.path.exists(model_weight_path), "cannot find {} file".format(model_weight_path)
    net.load_state_dict(torch.load(model_weight_path, map_location=device))
    net.to(device)

    # read class_indict
    assert os.path.exists(json_label_path), "cannot find {} file".format(json_label_path)
    json_file = open(json_label_path, 'r')
    class_indict = json.load(json_file)

    labels = [label for _, label in class_indict.items()]
    model_select = figs_directory
    confusion = cm.ConfusionMatrix(num_classes, labels=labels, model=model_select)
    net.eval()
    with torch.no_grad():
        for val_data in tqdm(validate_dataloader):
            val_images, val_labels = val_data
            outputs = net(val_images.to(device))
            outputs = torch.softmax(outputs, dim=1)
            outputs = torch.argmax(outputs, dim=1)
            confusion.update(outputs.to("cpu").numpy(), val_labels.to("cpu").numpy())
    confusion.plot_probability()
    # confusion.plot()
    # confusion.summary()


In [68]:
# 初始化
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(666)
torch.cuda.manual_seed(666)

# # 训练TENG&压力数据一起
# # 参数可选："1D-CNN", "ResCNN", "LSTM-FCN", "InceptionTime"
train_main(model_select="1D-CNN", inplanes=7, num_classes=3, batch_size=4, learning_rate=5*1e-2, epoch_max=1200)

# train_main(model_select="InceptionTime", inplanes=1, num_classes=10, batch_size=128, learning_rate=5*1e-2, epoch_max=40)

# train_main(model_select="ResCNN", inplanes=19, num_classes=12, batch_size=32, learning_rate=5*1e-2, epoch_max=100)

# train_main(model_select="LSTM-FCN", inplanes=1, num_classes=10, batch_size=128, learning_rate=5*1e-2, epoch_max=40)

# train_main(model_select="FR_CNN_Net", inplanes=1, num_classes=10 ,batch_size=64, learning_rate=5*1e-2, epoch_max=100)


(90, 7, 200)
(90,)
(90, 7, 200)
(90,)
(20, 7, 200)
train data size: 70 ,val data size: 20


100%|██████████| 18/18 [00:00<00:00, 166.44it/s]


#epoch: 00 
 train_loss: 1.257e+00 train_acc: 0.264 
 val_loss: 1.297e+00 val_acc: 0.250 time: 56



100%|██████████| 18/18 [00:00<00:00, 180.39it/s]


#epoch: 01 
 train_loss: 1.246e+00 train_acc: 0.222 
 val_loss: 1.147e+00 val_acc: 0.350 time: 56



...

#epoch: 1197 
 train_loss: 3.007e-02 train_acc: 0.986 
 val_loss: 4.650e+00 val_acc: 0.350 time: 56



100%|██████████| 18/18 [00:00<00:00, 169.02it/s]


#epoch: 1198 
 train_loss: 5.031e-02 train_acc: 0.986 
 val_loss: 5.578e+00 val_acc: 0.250 time: 56



100%|██████████| 18/18 [00:00<00:00, 203.14it/s]


#epoch: 1199 
 train_loss: 8.299e-02 train_acc: 0.958 
 val_loss: 8.223e+00 val_acc: 0.300 time: 56



In [69]:

# 打印网络结构
# 每次运行需要重启，清空之前缓存变量
# 姿态识别所用网络参数结构5*8
# models.print_1D_CNN_structure(2,10)
# models.print_ResCNN_net_structure(inplanes=2, num_classes=10)
# models.print_InceptionTime_net_structure(inplanes=2, num_classes=10)
# models.print_LSTM_FCN_net_structure(2, 10)
# models.print_FR_CNN_net_structure(inplanes=2, num_classes=10)



In [70]:

# 测试函数
# def test_main(

inplanes=7
num_classes=3
state_dict_file='./1D-CNN/Train_20231109-2228/model_20231109-2228/current_w.pth'
dataset_file='./1D-CNN/Train_20231109-2228/data_20231109-2228.pth'
model = models.Conv1dNet(inplanes, num_classes)
model = model.to(device)
model.load_state_dict(torch.load(state_dict_file)['state_dict'])
test_dataset = torch.load(dataset_file)['val']
test_label = torch.load(dataset_file)['label_val']
inputs = torch.Tensor(test_dataset).to(device)
output = F.log_softmax(model(inputs), dim=1)
pred = output.argmax(dim=1)
pred_new = pred.cpu().numpy()
# pred_count = []
# expand = 20
# for i in range(len(pred_new) // expand):
#     c = np.bincount(pred_new[i * expand:(i + 1) * expand])
#     pred_count.append(np.argmax(c))
#     print(pred_new[i * expand:(i + 1) * expand].tolist())
print(pred_new.tolist())
print(test_label)


# test_main(inplanes=7, num_classes=3,)

[1, 1, 1, 1, 1, 1, 1, 2, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 2]
[1. 0. 1. 0. 0. 2. 0. 1. 0. 1. 0. 2. 2. 0. 0. 2. 1. 0. 2. 0.]


In [71]:
print(torch.isnan(torch.Tensor(inputs)).any())
print(torch.isinf(torch.Tensor(X_train)).any())


tensor(False)
tensor(False)


In [72]:
nan_mask = torch.isnan(inputs)
nan_indices = torch.nonzero(nan_mask, as_tuple=False)

print(nan_indices)

tensor([], size=(0, 3), dtype=torch.int64)
